### Statistical Learning for Data Science 2 (229352) 
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #3

In [17]:
from sklearn.datasets import fetch_20newsgroups

train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

Xtrain = train.data
ytrain = train.target
Xtest = test.data
ytest = test.target

print("X:", len(Xtest))
print("y:", len(ytest))

X: 7532
y: 7532


In [18]:
print("X[0]:", Xtrain[0])
print("y[0]:", ytrain[0])

X[0]: From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----





y[0]: 7


In [19]:
train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

### Apply Tfidf ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html))

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words = 'english',
                        ngram_range = (1, 1))
Xtrain_tfidf = tfidf.fit_transform(Xtrain)

Xtrain_tfidf

<11314x129796 sparse matrix of type '<class 'numpy.float64'>'
	with 1300729 stored elements in Compressed Sparse Row format>

In [29]:
#tfidf.vocabulary_

### Classify with Naive Bayes ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html))

In [30]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB(alpha=1)
nb.fit(Xtrain_tfidf, ytrain)

MultinomialNB(alpha=1)

Evaluate on the test set using [`classification_report`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) 

We will focus on the [F1-score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)

In [31]:
from sklearn.metrics import classification_report

Xtest_tfidf = tfidf.transform(Xtest)

ypred = nb.predict(Xtest_tfidf)

print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.80      0.69      0.74       319
           1       0.78      0.72      0.75       389
           2       0.79      0.72      0.75       394
           3       0.68      0.81      0.74       392
           4       0.86      0.81      0.84       385
           5       0.87      0.78      0.82       395
           6       0.87      0.80      0.83       390
           7       0.88      0.91      0.90       396
           8       0.93      0.96      0.95       398
           9       0.91      0.92      0.92       397
          10       0.88      0.98      0.93       399
          11       0.75      0.96      0.84       396
          12       0.84      0.65      0.74       393
          13       0.92      0.79      0.85       396
          14       0.82      0.94      0.88       394
          15       0.62      0.96      0.76       398
          16       0.66      0.95      0.78       364
          17       0.95    

### Combine all methods into a [pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('tfidf', TfidfVectorizer(stop_words = 'english')),
                     ('nb', MultinomialNB())])

pipeline.fit(Xtrain, ytrain)
ypred = pipeline.predict(Xtest)
print(classification_report(ytest, ypred))

Now we will use [grid search cross-validation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to find model with the best hyperparameters

![5CV](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'tfidf__ngram_range': [(1, 1), (1, 2), (2, 2)],
          'nb__alpha': [0.01, 0.1, 1, 10]}

gridcv = GridSearchCV(pipeline, params,
                      scoring='f1_macro', cv=5)
gridcv.fit(Xtrain, ytrain)

In [ ]:
gridcv.best_estimator_

In [ ]:
ypred = gridcv.predict(Xtest)
print(classification_report(ytest, ypred))

#### Exercise

1. For the Naive Bayes model, use grid search cross-validation across different values of `alpha` and `ngram_range` to find the best model.

2. For the best value of `alpha` and `ngram_range`, compute the `f1_macro` score on the test set. 
* What value of `alpha` and `ngram_range` did you choose?
* What is the model's `f1_macro` score?

Extra: Try `GaussianNB` [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html). Do you get a better result?